In [6]:
import pandas as pd
import matplotlib as plt

In [18]:
transactions = pd.read_csv("./Data/amazon-purchases.csv")

In [ ]:
print(transactions.head())

   Order Date  Purchase Price Per Unit  Quantity Shipping Address State  \
0  2018-12-04                     7.98       1.0                     NJ   
1  2018-12-22                    13.99       1.0                     NJ   
2  2018-12-24                     8.99       1.0                     NJ   
3  2018-12-25                    10.45       1.0                     NJ   
4  2018-12-25                    10.00       1.0                     NJ   

                                               Title ASIN/ISBN (Product Code)  \
0  SanDisk Ultra 16GB Class 10 SDHC UHS-I Memory ...               B0143RTB1E   
1  Betron BS10 Earphones Wired Headphones in Ear ...               B01MA1MJ6H   
2                                                NaN               B078JZTFN3   
3  Perfecto Stainless Steel Shaving Bowl. Durable...               B06XWF9HML   
4                      Proraso Shaving Cream for Men               B00837ZOI0   

        Category  Survey ResponseID  
0   FLASH_MEMORY  R_01vN

In [33]:
transactions["Order Date"] = pd.to_datetime(transactions["Order Date"], format = "%y%m%d")


print(transactions.dtypes)

Order Date                  datetime64[ns]
Purchase Price Per Unit            float64
Quantity                           float64
Shipping Address State              object
Title                               object
ASIN/ISBN (Product Code)            object
Category                            object
Survey ResponseID                   object
dtype: object


In [38]:
uniq_cat = transactions["Category"].unique()
len(uniq_cat)

1872